In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('../data/movies.csv')
ratings = pd.read_csv('../data/ratings.csv')

In [3]:
# Cuáles fueron los géneros mejor rankeados en promedio en el año 2005?
movies_2005 = movies[movies.title.str.contains('2005', case=False)]
ratings_2005 = ratings[ratings.movieId.isin(movies_2005.movieId)]
ratings_2005 = ratings_2005.merge(movies, on='movieId').drop(columns=['movieId', 'userId', 'timestamp', 'title']).set_index('genres').stack().reset_index()
ratings_2005.columns = ['genres', 'type', 'rating']
ratings_2005 = ratings_2005.groupby('genres').rating.mean().sort_values(ascending=False)
ratings_2005

genres
Animation|Children|Comedy|Fantasy                  5.000
Comedy|War                                         5.000
Comedy|Drama|Musical                               4.750
Animation|Children|Comedy|Romance                  4.500
Comedy|Documentary|Romance                         4.500
                                                   ...  
Adventure|Comedy                                   1.875
Adventure|Animation|Children|Comedy|Fantasy|War    1.500
Adventure|Children|Comedy|Fantasy                  1.000
Action|Horror|Sci-Fi|Thriller                      0.500
Action|Adventure|Fantasy|Thriller                  0.500
Name: rating, Length: 117, dtype: float64

In [4]:
# Cuál es el top 5 de películas mejor rankeadas en promedio?
top5_ranking = ratings.merge(movies, on='movieId').drop(columns=['movieId', 'userId', 'timestamp', 'genres'])
top5_ranking = top5_ranking.groupby('title').rating.mean().sort_values(ascending=False)
top5_ranking.head(5)


title
Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990)    5.0
Meantime (1984)                                            5.0
Decalogue, The (Dekalog) (1989)                            5.0
Stand, The (1994)                                          5.0
Big Top Scooby-Doo! (2012)                                 5.0
Name: rating, dtype: float64

In [5]:
# Cuál es el género con más reviews?
top1_genre = ratings.merge(movies, on='movieId').drop(columns=['movieId', 'userId', 'timestamp', 'rating'])
top1_genre = top1_genre.set_index('genres').stack().reset_index()
top1_genre.columns = ['genres', 'type', 'rating']
top1_genre = top1_genre.groupby('genres').rating.count().sort_values(ascending=False)
top1_genre.head(1)

genres
Comedy    7196
Name: rating, dtype: int64

In [6]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

In [7]:
# Cuáles es el género por año con mejor rating entre el año 2000 y 2010?
movies_2000_2010 = movies[(movies['year'] >= '2000') & (movies['year'] <= '2010')]
ratings_2000_2010 = ratings[ratings.movieId.isin(movies_2000_2010.movieId)]
ratings_2000_2010 = ratings_2000_2010.merge(movies, on='movieId').drop(columns=['movieId', 'userId', 'timestamp', 'title','year']).set_index('genres').stack().reset_index()
ratings_2000_2010.columns = ['genres', 'type', 'rating']
ratings_2000_2010 = ratings_2000_2010.groupby(['genres', 'type']).rating.mean().sort_values(ascending=False)
ratings_2000_2010

genres                              type  
Comedy|War                          rating    5.000000
Action|Comedy|Drama|Romance         rating    5.000000
Comedy|Crime|Drama|Horror           rating    5.000000
Animation|Drama|Fantasy|Mystery     rating    5.000000
Action|Adventure|Animation|Fantasy  rating    5.000000
                                                ...   
Action|Comedy|Crime|Western         rating    1.000000
Comedy|Horror|Sci-Fi|Thriller       rating    1.000000
Musical|Romance                     rating    0.833333
Drama|Fantasy|Sci-Fi|Thriller       rating    0.500000
Action|Comedy|Horror|Musical        rating    0.500000
Name: rating, Length: 531, dtype: float64

In [8]:
# Cuál es el usuario con más críticas y cuál es su género favorito?
top1_user = ratings.merge(movies, on='movieId').drop(columns=['movieId', 'timestamp', 'rating'])
top1_user = top1_user.set_index('userId').stack().reset_index()
top1_user.columns = ['userId', 'type', 'genres']
top1_user = top1_user.groupby('userId').genres.count().sort_values(ascending=False)
top1_user = top1_user.head(1)
top1_user


userId
414    8093
Name: genres, dtype: int64

In [20]:
# Cuál es el género con más críticas por usuario?
top1_genre_user = ratings.merge(movies, on='movieId').drop(columns=['movieId', 'timestamp', 'rating'])
top1_genre_user = top1_genre_user.set_index('userId').stack().reset_index()
top1_genre_user.columns = ['userId', 'type', 'genres']
top1_genre_user = top1_genre_user.groupby(['userId', 'genres']).type.count().sort_values(ascending=False)
top1_genre_user = top1_genre_user.groupby('userId').head(1)
top1_genre_user = top1_genre_user.groupby('genres').count().sort_values(ascending=False)
top1_genre_user.head(1)

genres
1995    77
Name: type, dtype: int64